In [ ]:
!python -m pip install turicreate
!python -m jupyter notebook

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 92.0 MB 11 kB/s 
     |████████████████████████████████| 86.4 MB 57 kB/s 
     |████████████████████████████████| 3.5 MB 36.8 MB/s 
     |████████████████████████████████| 322 kB 52.4 MB/s 
     |████████████████████████████████| 3.6 MB 38.2 MB/s 
     |████████████████████████████████| 18.3 MB 297 kB/s 
     |████████████████████████████████| 449 kB 36.4 MB/s 
     |████████████████████████████████| 3.8 MB 35.6 MB/s 
     |████████████████████████████████| 2.9 MB 42.0 MB/s 
     |████████████████████████████████| 20.1 MB 1.1 MB/s 
     |████████████████████████████████| 50 kB 7.6 MB/s 
  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13714 sha256=310dc24c0aed0dc9eb1ad2cec9278929759a0b5c2dcdd595e043d8729580496c
  Stored in directory: /root/.cache/pip/wheels/b2/7f/f6/f180315b584f00445045ff1699b550fa895d09471337ce21c6


In [ ]:
import turicreate as tc
import pandas as pd

In [ ]:
books = pd.read_csv('ratings.csv')
books.shape

(5976479, 3)

In [ ]:
max(books['user_id'])

53425

In [ ]:
# Add books to the df
new_ratings = pd.read_csv('new_ratings.csv')
books = pd.concat([books, new_ratings])
books.shape

(5976486, 3)

In [ ]:
# books = tc.SFrame.read_csv("ratings.csv", header=True,delimiter=',')
books = tc.SFrame(books)
# books

In [ ]:
books.show()

Materializing SFrame

In [ ]:
training_data, validation_data = tc.recommender.util.random_split_by_user(books, 'user_id', 'book_id',item_test_proportion=0.3)


In [ ]:
print(training_data.shape)
print(validation_data.shape)

(5942664, 3)
(33822, 3)


In [ ]:
# Item-item collaborative filtering - cosine similarity
model = tc.recommender.item_similarity_recommender.create(training_data,
user_id='user_id', item_id='book_id',target='rating', similarity_type = 'cosine') 
# items_similarity = model.get_similar_items()

Preparing data set.

Data has 5942664 observations with 53425 users and 10000 items.

Data prepared in: 7.86111s

Training model from provided data.

Gathering per-item and per-user statistics.

...

| 23.87s                              | 100              | 10000           |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 24.3244s

In [ ]:
results = model.recommend(users = [53425], k=3, verbose = False)

In [ ]:
books_info = pd.read_csv('books_enriched.csv', index_col = 'book_id')

In [ ]:
# books_info.loc[results['book_id'][-3:], 'title']
books_info.loc[results['book_id'], 'title']

book_id
3                     Twilight (Twilight, #1)
1     The Hunger Games (The Hunger Games, #1)
96      Fifty Shades Freed (Fifty Shades, #3)
Name: title, dtype: object

In [ ]:
model.evaluate(validation_data)
#Item based Collaborative filtering Overall RMSE (Cosine Similarity): 3.9038129692577996

recommendations finished on 1000/1000 queries. users per second: 1873.86


Precision and recall summary statistics by cutoff
+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    |  0.5130000000000002 | 0.016576138439357724 |
|   2    |  0.4970000000000002 | 0.03180093855864954  |
|   3    |  0.4693333333333332 | 0.044549700932059726 |
|   4    |       0.44675       | 0.05611583857622838  |
|   5    | 0.42859999999999987 | 0.06693753260915429  |
|   6    | 0.41583333333333306 | 0.07745808071135243  |
|   7    |        0.402        | 0.08727532744485918  |
|   8    |       0.38825       | 0.09628049972567387  |
|   9    | 0.37888888888888883 | 0.10538092330522317  |
|   10   | 0.36679999999999985 | 0.11285791392007859  |
+--------+---------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 3.9034793500600307

Per User RMSE (best)
+---------+-------------------+-------+
| user_id |        rmse       | count |
+-----

{'precision_recall_by_user': Columns:
 	user_id	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 18000
 
 Data:
 +---------+--------+--------------------+----------------------+-------+
 | user_id | cutoff |     precision      |        recall        | count |
 +---------+--------+--------------------+----------------------+-------+
 |    61   |   1    |        1.0         | 0.022222222222222223 |   45  |
 |    61   |   2    |        1.0         | 0.044444444444444446 |   45  |
 |    61   |   3    |        1.0         | 0.06666666666666667  |   45  |
 |    61   |   4    |        0.75        | 0.06666666666666667  |   45  |
 |    61   |   5    |        0.6         | 0.06666666666666667  |   45  |
 |    61   |   6    | 0.6666666666666666 | 0.08888888888888889  |   45  |
 |    61   |   7    | 0.5714285714285714 | 0.08888888888888889  |   45  |
 |    61   |   8    |       0.625        |  0.1111111111111111  |   45  |
 |    61   |   9    | 0.5555555555555556 |  0.111111

In [ ]:
# Item-item collaborative filtering - correlation
model = tc.recommender.item_similarity_recommender.create(training_data,
user_id='user_id', item_id='book_id',target='rating', similarity_type = 'pearson') 

Preparing data set.

Data has 5942664 observations with 53425 users and 10000 items.

Data prepared in: 8.11225s

Training model from provided data.

Gathering per-item and per-user statistics.

...

| 34.11s                              | 100              | 10000           |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 34.5542s

In [ ]:
results = model.recommend(users = [53425], k=3, verbose = False)
# books_info.loc[results['book_id'][-3:], 'title']
books_info.loc[results['book_id'], 'title']

book_id
3628                       The Complete Calvin and Hobbes
7947                                      ESV Study Bible
9566    Attack of the Deranged Mutant Killer Monster S...
Name: title, dtype: object

In [ ]:
model.evaluate(validation_data)
#Item based Collaborative filtering Overall RMSE (Correlation): 0.9485244028004367

recommendations finished on 1000/1000 queries. users per second: 1743.37


Precision and recall summary statistics by cutoff
+--------+-----------------------+------------------------+
| cutoff |     mean_precision    |      mean_recall       |
+--------+-----------------------+------------------------+
|   1    | 0.0010000000000000002 | 2.1739130434782607e-05 |
|   2    | 0.0005000000000000001 | 2.1739130434782607e-05 |
|   3    | 0.0006666666666666666 | 5.204216073781291e-05  |
|   4    | 0.0007500000000000002 | 8.234519104084322e-05  |
|   5    | 0.0009999999999999994 | 0.0001412196499153021  |
|   6    | 0.0009999999999999996 | 0.0001745529832486354  |
|   7    | 0.0011428571428571427 | 0.0002334274421230942  |
|   8    | 0.0015000000000000005 |  0.000340540518748366  |
|   9    | 0.0016666666666666663 | 0.0004377683754082714  |
|   10   | 0.0017000000000000003 | 0.0004944166838268392  |
+--------+-----------------------+------------------------+
[10 rows x 3 columns]


Overall RMSE: 0.9507324696758243

Per User RMSE (best)
+---------+-------------------

{'precision_recall_by_user': Columns:
 	user_id	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 18000
 
 Data:
 +---------+--------+-----------+--------+-------+
 | user_id | cutoff | precision | recall | count |
 +---------+--------+-----------+--------+-------+
 |    61   |   1    |    0.0    |  0.0   |   45  |
 |    61   |   2    |    0.0    |  0.0   |   45  |
 |    61   |   3    |    0.0    |  0.0   |   45  |
 |    61   |   4    |    0.0    |  0.0   |   45  |
 |    61   |   5    |    0.0    |  0.0   |   45  |
 |    61   |   6    |    0.0    |  0.0   |   45  |
 |    61   |   7    |    0.0    |  0.0   |   45  |
 |    61   |   8    |    0.0    |  0.0   |   45  |
 |    61   |   9    |    0.0    |  0.0   |   45  |
 |    61   |   10   |    0.0    |  0.0   |   45  |
 +---------+--------+-----------+--------+-------+
 [18000 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, num_columns=n) to print more rows and colum

In [ ]:
# Matrix factorization
model = tc.recommender.ranking_factorization_recommender.create(training_data,
user_id='user_id', item_id='book_id',target='rating')
results = model.recommend(users = [53425], k=3, verbose = False)

Preparing data set.

Data has 5942664 observations with 53425 users and 10000 items.

Data prepared in: 7.61085s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.77435

Final training RMSE: 0.69848

In [ ]:
# books_info.loc[results['book_id'][-3:], 'title']
books_info.loc[results['book_id'], 'title']

book_id
10      Pride and Prejudice
4     To Kill a Mockingbird
31                 The Help
Name: title, dtype: object

In [ ]:
# results[-3:]
model.recommend(users = [53425], k=3, verbose = False)

user_id,book_id,score,rank
53425,10,3.910995824815994,1
53425,4,3.889518542768722,2
53425,31,3.883325918199783,3


In [ ]:
model.evaluate(validation_data)
#Matrix factorization Overall RMSE: 0.9895585838230585

recommendations finished on 1000/1000 queries. users per second: 1771.05


Precision and recall summary statistics by cutoff
+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    | 0.16999999999999998 | 0.005094389543216736 |
|   2    | 0.17399999999999996 | 0.010508896138885126 |
|   3    | 0.17166666666666675 | 0.015657707324115804 |
|   4    | 0.16524999999999998 | 0.020143091132116053 |
|   5    |  0.1612000000000001 | 0.02451749883265509  |
|   6    | 0.15833333333333333 | 0.028872264204611354 |
|   7    | 0.15728571428571425 | 0.03338158636247439  |
|   8    |       0.154625      | 0.03761007762479203  |
|   9    | 0.15311111111111092 | 0.041848728132836105 |
|   10   | 0.15209999999999999 | 0.046144071607848995 |
+--------+---------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.9909650433338858

Per User RMSE (best)
+---------+--------------------+-------+
| user_id |        rmse        | count |
+---

{'precision_recall_by_user': Columns:
 	user_id	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 18000
 
 Data:
 +---------+--------+---------------------+----------------------+-------+
 | user_id | cutoff |      precision      |        recall        | count |
 +---------+--------+---------------------+----------------------+-------+
 |    61   |   1    |         0.0         |         0.0          |   45  |
 |    61   |   2    |         0.0         |         0.0          |   45  |
 |    61   |   3    |  0.3333333333333333 | 0.022222222222222223 |   45  |
 |    61   |   4    |         0.25        | 0.022222222222222223 |   45  |
 |    61   |   5    |         0.2         | 0.022222222222222223 |   45  |
 |    61   |   6    | 0.16666666666666666 | 0.022222222222222223 |   45  |
 |    61   |   7    | 0.14285714285714285 | 0.022222222222222223 |   45  |
 |    61   |   8    |        0.125        | 0.022222222222222223 |   45  |
 |    61   |   9    |  0.1111111111111111